## 🔐 Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI:

```bash
az login
```

# 🏦 Custom Chat Message Store for Threads

## Industry Use Case: Compliance-Ready Conversation Audit Trail

This notebook demonstrates how to implement a **custom chat message store** for managing conversation threads with compliance and audit requirements.

| Feature | FSI Application |
|---------|----------------|
| **Custom Storage** | Store conversations in compliance-approved databases |
| **Full Serialization** | Complete message history for regulatory audits |
| **Data Control** | Meet data residency and retention requirements |
| **Flexible Backend** | Integrate with existing enterprise systems |

In [ ]:
import os
from typing import Any, Collection
from azure.identity import AzureCliCredential
from agent_framework import ChatMessage, ChatMessageStoreProtocol
from agent_framework.azure import AzureOpenAIChatClient
from dotenv import load_dotenv

load_dotenv("../../.env")
print("✅ Environment loaded and dependencies imported")

## Implement Custom Chat Message Store

This store could integrate with compliance-approved databases (PostgreSQL, Cosmos DB, etc.) for regulatory audit requirements.

In [ ]:
class CustomChatMessageStore(ChatMessageStoreProtocol):
    """Custom chat message store for compliance-ready conversation storage.
    
    In production, this would integrate with:
    - Relational databases (PostgreSQL, SQL Server)
    - NoSQL databases (Cosmos DB, MongoDB)
    - Enterprise storage systems with audit capabilities
    """

    def __init__(self, messages: Collection[ChatMessage] | None = None) -> None:
        self._messages: list[ChatMessage] = []
        if messages:
            self._messages.extend(messages)

    async def add_messages(self, messages: Collection[ChatMessage]) -> None:
        """Add messages to the store (would write to database in production)."""
        self._messages.extend(messages)

    async def list_messages(self) -> list[ChatMessage]:
        """Retrieve all messages (would query database in production)."""
        return self._messages

    @classmethod
    async def deserialize(cls, serialized_store_state: Any, **kwargs: Any) -> "CustomChatMessageStore":
        """Create a new instance from serialized state."""
        store = cls()
        await store.update_from_state(serialized_store_state, **kwargs)
        return store

    async def update_from_state(self, serialized_store_state: Any, **kwargs: Any) -> None:
        """Update this instance from serialized state."""
        if serialized_store_state:
            state = ChatMessageStoreState.from_dict(serialized_store_state, **kwargs)
            if state.messages:
                self._messages.extend(state.messages)

    async def serialize(self, **kwargs: Any) -> Any:
        """Serialize this store's state."""
        state = ChatMessageStoreState(messages=self._messages)
        return state.to_dict(**kwargs)


print("✅ CustomChatMessageStore defined")

## Define Compliance Officer Tools

In [ ]:
async def check_transaction_compliance(transaction_id: str) -> str:
    """Check if a transaction meets compliance requirements."""
    compliance_data = {
        "TXN-001": {"status": "Compliant", "risk_level": "Low", "aml_check": "Passed"},
        "TXN-002": {"status": "Under Review", "risk_level": "High", "aml_check": "Pending"},
        "TXN-003": {"status": "Flagged", "risk_level": "Critical", "aml_check": "Failed"},
    }
    c = compliance_data.get(transaction_id, {"status": "Unknown", "risk_level": "Unknown", "aml_check": "Unknown"})
    return f"Transaction {transaction_id}: Status={c['status']}, Risk={c['risk_level']}, AML Check={c['aml_check']}"


async def get_regulatory_requirements(regulation_type: str) -> str:
    """Get regulatory requirements for a specific regulation."""
    regulations = {
        "AML": "Anti-Money Laundering: Customer verification, transaction monitoring, suspicious activity reporting",
        "KYC": "Know Your Customer: Identity verification, risk assessment, ongoing monitoring",
        "GDPR": "Data Protection: Consent management, data minimization, right to erasure",
    }
    return regulations.get(regulation_type, f"No specific requirements found for {regulation_type}")


print("✅ Tools defined: check_transaction_compliance, get_regulatory_requirements")

## Run Compliance Officer with Custom Store

The agent uses `chat_message_store_factory` to create custom store instances for each thread.

In [ ]:
async def run_compliance_officer():
    """Run compliance officer with custom message store."""
    print("\n--- Compliance Officer with Custom Message Store ---\n")
    
    # Note: Custom message stores require in-memory chat clients (not service-backed)
    # AzureOpenAIChatClient uses Azure OpenAI API with local message management
    # This is different from AzureAIAgentClient which uses service-managed threads
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o"))
    
    print(f"Using Azure OpenAI endpoint: {endpoint}")
    print(f"Using deployment: {deployment_name}\n")
    
    agent = AzureOpenAIChatClient(
        endpoint=endpoint,
        deployment_name=deployment_name,
        credential=AzureCliCredential(),
    ).as_agent(
        name="ComplianceOfficer",
        instructions="You are a compliance officer helping with regulatory inquiries. Check transactions and provide regulatory guidance.",
        tools=[check_transaction_compliance, get_regulatory_requirements],
        chat_message_store_factory=CustomChatMessageStore,  # Use custom store
    )
    
    # Start new thread (creates CustomChatMessageStore instance)
    thread = agent.get_new_thread()
    print("✅ Thread created with custom message store\n")
    
    # Phase 1: Initial conversation
    print("=" * 60)
    print("Phase 1: Compliance Review Session")
    print("=" * 60)
    
    query1 = "Please check compliance status for transaction TXN-002"
    print(f"Analyst: {query1}")
    response1 = await agent.run(query1, thread=thread)
    print(f"Officer: {response1.text}\n")
    
    # Phase 2: Serialize thread (full message history)
    print("=" * 60)
    print("Phase 2: Saving Compliance Session (Full History)")
    print("=" * 60)
    
    serialized_thread = await thread.serialize()
    
    print(f"\n💾 Serialized thread: {serialized_thread}\n")
    print("📊 Note: Custom stores serialize FULL message history")
    print("💡 This enables complete audit trails for compliance\n")
    
    # Phase 3: Resume session
    print("=" * 60)
    print("Phase 3: Resuming Compliance Session")
    print("=" * 60)
    
    resumed_thread = await agent.deserialize_thread(serialized_thread)
    print("\n✅ Session restored with full conversation history\n")
    
    query2 = "What do you remember about me?"
    print(f"Analyst: {query2}")
    response2 = await agent.run(query2, thread=resumed_thread)
    print(f"Officer: {response2.text}\n")
    
    print("=" * 60)
    print("✅ Custom message store demo completed!")
    print("=" * 60)

await run_compliance_officer()

## Key Takeaways

| Feature | Description |
|---------|-------------|
| `ChatMessageStoreProtocol` | Interface for custom storage backends |
| `chat_message_store_factory` | Factory to create store instances per thread |
| `serialize()` / `deserialize()` | Store state for compliance audit trails |

## Production Considerations

| Aspect | Implementation |
|--------|---------------|
| **Database** | PostgreSQL, Cosmos DB, SQL Server |
| **Audit Trail** | Immutable logs with timestamps |
| **Encryption** | Encrypt sensitive conversation data |
| **Retention** | Implement data retention policies |